In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# 3 프로젝트 모델 구현

## 1 Data Loader 구현

모델을 구현하기에 앞서 데이터를 가져올 Data Loader를 구현한다.

### 1 구현할 메서드 살펴보기

1. len

batch가 몇 개 있는지 확인하는 메서드이다.

In [ ]:
def __len__(self):
    return math.ceil(len(self.x) / self.batch_size)

2. getitem

batch를 생성하는 메서드이다.

In [ ]:
def __getitem__(self, idx):
    strt = idx * self.batch_size
    fin = (idx + 1) * self.batch_size
    data = self.df.iloc[strt:fin]

    batch_x, batch_y = self.get_data(data)

    return np.array(batch_x), np.array(batch_y)

In [ ]:
# import gdown
# url = 'https://drive.google.com/file/d/1USSTL8IlR_gux_FAPFI4wpOkEdanxgPZ'
# fname = 'oxford_pet.zip'
# gdown.download(url, fname, quiet=False)
# !unzip -qq /content/drive/MyDrive/oxford_pet.zip

In [ ]:
# !unzip /content/drive/MyDrive/oxford_pet.zip -d /content/

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style('whitegrid')

In [ ]:
df = pd.read_csv('./annotations/list.txt', skiprows=6, delimiter=' ', header=None)

# list.txt 안의 정보대로 파일 이름, id, species, breed 순서로 칼럼을 만들어 준다.
df.columns = ['file_name', 'id', 'species', 'breed']
df

### 2 Data Loader 구현하기

대용량 데이터 세트를 배치 처리할 수 있도록 Data Loader를 구현해 보자.

1. 라이브러리 import하기

필요한 라이브러리를 불러온다.

In [ ]:
import math
import random

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras

2. Sequence 작성하기

순서대로 코드를 작성해 보자.

In [ ]:
class DataGenerator(keras.utils.Sequence):
# 객체 생성 시 필요한 인자 정하기
# 배치 사이즈, 데이터를 폴더별로 저장했기 때문에 csv 파일 경로
# fold
# 학습인지를 다지는 mode
    def __init__(self, batch_size, csv_path,
                 fold, image_size, mode='train',
                 shuffle=True):
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.fold = fold
        self.mode = mode
        self.image_size = image_size # Added missing image_size assignment

        # csv 파일 경로 가져오기
        self.df = pd.read_csv(csv_path)
# mode에 따른 fold 나누기
        if self.mode == 'train':
            self.df = self.df[self.df['fold'] != self.fold]
        elif self.mode == 'val':
            self.df = self.df[self.df['fold'] == self.fold]
# 한 번씩 데이터 셔플해 주기
        self.on_epoch_end()
# batch가 몇 개 있는지 확인하기, 남는 데이터를 위해 올림 사용
    def __len__(self):
        return math.ceil(len(self.df) / self.batch_size)

# batch의 index를 input으로 받는다.
# 예시 코드를 활용하여 작성한다.

    def __getitem__(self, idx):
        strt = idx * self.batch_size
        fin = (idx + 1) * self.batch_size
        data = self.df.iloc[strt:fin]

        # get_data 함수 호출해 저장
        batch_x, batch_y = self.get_data(data)

        return np.array(batch_x), np.array(batch_y)

# 데이터 안의 모든 이미지와 label을 읽어 온다.
    def get_data(self, data):
        batch_x = []
        batch_y = []

        for _, r in data.iterrows():
            file_name = r['file_name']
            # 경로에서 가져오기
            image_path = f'./images/{file_name}.jpg'
            image = cv2.imread(image_path)

            # Check if the image was loaded successfully
            if image is None:
                print(f"Warning: Could not load image from {image_path}. Skipping.")
                continue # Skip this iteration if image loading failed

            # BGR -> RGB로 변경
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            # 사이즈 재지정
            image = cv2.resize(image, (self.image_size, self.image_size))
            # 이미지 픽셀을 0~255로 재지정
            image = image / 255.

            # 해당 이미지가 개인지, 고양인지 이진 분류(고양이 0, 강아지 1로 재지정)

            label = int(r['species']) -1

            batch_x.append(image)
            batch_y.append(label)

        return batch_x, batch_y
# 콜백 메서드 작성, 셔플해 주는 함수
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

3. 동작 확인하기

DataGenerator 객체를 생성해 보자.

In [ ]:
csv_path = './kfolds.csv'
train_generator = DataGenerator(
# 시각화하려고 너무 크지 않게 9로 지정
    batch_size=9,
    csv_path=csv_path,
# 첫 번째 fold로
    fold=1,
# 256 * 256로 구현
    image_size=256,
    mode='train',
    shuffle=True
)
# 배치 사이즈가 9이므로 다시 9를 곱해 주면 5886이 나오는 것을 확인할 수 있다.
print(len(train_generator)*9)

4. 불러오기

개와 고양이를 잘 분류하는지 이미지를 출력해 보자.

In [ ]:
class_name = ['Cat', 'Dog']

for batch in train_generator:
    X, y = batch
    plt.figure(figsize=(15,15))
    # 9개 배치이므로 9개를 가져온다.
    for i in range(9):
        ax = plt.subplot(3, 3, i+1)
        plt.imshow(X[i])
    # 각 타이틀에 맞게 개인지 고양이인지 적어 준다.
        plt.title(class_name[y[i]])
        plt.axis('off')

    # 첫 번째 배치만 확인하고 종료
    break

## 2 모델 구현

모델 구현에 가장 많이 사용하고 있는 케라스의 Sequence를 활용해 구현하고자 한다. 1개의 input과 1개의 output을 구현하기에 적합하다.

### 1 구현할 메서드

모델 구현에 앞서 구현할 메서드를 살펴보자.

1. 라이브러리 가져오기

필요한 라이브러리를 가져오고 사용할 GPU를 지정한다.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations

import os
os.environ['CUDA_VISIBLE_DEVICE'] = '1'

1. get_sequential_model

Sequential 모델은 순차적으로 레이어 층을 더해 주는 모델이다.

In [ ]:
def get_sequential_model(input_shape):
    model = keras.Sequential(
        [ layers.Input(input_shape), # input
          # 1st Conv block
          layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
          layers.conv2D(64, 3, strides=1, activation='relu', padding='same'),
          layers.MaxPool2D(),
          layers.BatchNormalization(),
          layers.Dropout(0.5),
          # 2nd Conv block
          layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
          layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
          layers.MaxPool2D(),
          layers.BatchNormalization(),
          layers.Dropout(0.3),
          # Classifier
          layers.GlobalMaxPool2D(),
          layers.Dense(128, activation='relu'),
          layers.Dense(1, activation='sigmoid')])

### 2 Sequence 작성하기

순서대로 코드를 작성해 보자.

1. 모델 생성하기

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations

import os
os.environ['CUDA_VISIBLE_DEVICE'] = '1'

def get_sequential_model(input_shape):
    model = keras.Sequential(
        [ layers.Input(input_shape), # input
          # 1st Conv block
          layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
          layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
          layers.MaxPool2D(),
          layers.BatchNormalization(),
          layers.Dropout(0.5),
          # 2nd Conv block
          layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
          layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
          layers.MaxPool2D(),
          layers.BatchNormalization(),
          layers.Dropout(0.3),
          # Classifier
          layers.GlobalMaxPool2D(),
          layers.Dense(128, activation='relu'),
          layers.Dense(1, activation='sigmoid')])

    return model

# 모양은 256x256x3(RGB)
input_shape = (256, 256, 3)
model = get_sequential_model(input_shape)

model.summary()

2. 학습 및 평가에 사용할 내용 정하기

모델의 최적화 함수, 손실 함수, 측정 항목을 설정한다.

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'] # Changed to a list
)

## 3 모델 학습

구현한 모델을 가지고 실제로 학습해 본다.

### 1 코드 작성하기

1. 라이브러리 삽입 및 데이터 로드 코드 작성하기

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations
import pandas as pd # Import pandas
import cv2 # Import cv2
import math # Import math
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

class DataGenerator(keras.utils.Sequence):
    # Corrected the typo from __init_ to __init__
    def __init__(self, batch_size, csv_path, image_size,
               fold, mode='train', shuffle=True):
        self.batch_size =batch_size
        self.image_size = image_size
        self.fold = fold
        self.mode = mode
        self.shuffle = shuffle # Add shuffle attribute assignment

        self.df = pd.read_csv(csv_path)

        if self.mode == 'train':
            self.df = self.df[self.df['fold'] != self.fold]
        elif self.mode == 'val':
            self.df = self.df[self.df['fold'] == self.fold]

        self.on_epoch_end()

    def __len__(self):
        return math.ceil(len(self.df) / self.batch_size)

    def __getitem__(self, idx):
        strt = idx * self.batch_size
        fin = (idx + 1) * self.batch_size
        data = self.df.iloc[strt:fin]

        batch_x, batch_y = self.get_data(data)

        # Ensure returned data is float32 for common model inputs
        return np.array(batch_x, dtype=np.float32), np.array(batch_y, dtype=np.float32)


    def get_data(self, data):
        batch_x = []
        batch_y = []

        for _, r in data.iterrows(): # Corrected typo from iterrow to iterrows
            file_name = r['file_name']

            # Corrected typo from .jgp to .jpg and added check for image loading
            image_path = f'./images/{file_name}.jpg'
            image = cv2.imread(image_path)

            # Add a check to ensure the image was loaded successfully
            if image is None:
                print(f"Warning: Could not load image from {image_path}. Skipping.")
                continue # Skip this iteration if image loading failed

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            image = cv2.resize(image, (self.image_size, self.image_size))

            # Scale pixel values to be between 0 and 1
            image = image / 255.0

            label = int(r['species']) - 1

            batch_x.append(image)
            batch_y.append(label)

        return batch_x, batch_y

    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

2. 훈련 데이터 및 평가 데이터 지정하기

In [ ]:
# csv_path = './kfolds.csv'
# train_generator = DataGenerator(
#     fold=1, mode='train', csv_path=csv_path, batch_size=128,
#     image_size=256, shuffle=True)

# valid_generator = DataGenerator(
#     fold=1, mode = 'val', csv_path=csv_path,
#     batch_size=128, image_size=256, shuffle=True)

In [ ]:
csv_path = './kfolds.csv'
train_generator = DataGenerator(
    fold=1, mode='train', csv_path=csv_path, batch_size=64,  # Reduced batch size
    image_size=256, shuffle=True)

valid_generator = DataGenerator(
    fold=1, mode = 'val', csv_path=csv_path,
    batch_size=64, image_size=256, shuffle=True) # Reduced batch size

3. 학습하기

훈련 데이터는 70% 정확도, 검증 데이터는 68% 정도의 정확도가 나왔다.

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=3,
    verbose=1
)

4. 훈련 결과 시각화하기

훈련 데이터 및 검증 데이터의 정확도를 시각화하여 확인할 수 있다.

In [ ]:
import matplotlib.pyplot as plt
history = history.history

plt.figure(figsize=(15, 5))
plt.subplot(1,2,1)
plt.plot(history['loss'], label='train')
plt.plot(history['val_loss'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title("Loss")

plt.subplot(1,2,2)
plt.plot(history['accuracy'], label='train')
plt.plot(history['val_accuracy'], label='val')
plt.legend()

plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title("Accuracy")
plt.show()